In [ ]:
#default_exp prodigy_demo

In [ ]:
#no_test

#data structure imports
import pandas as pd
import numpy as np

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *

In [ ]:
#no_test
seed = 2436
random.seed(seed)
fix_random_seed(seed)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/matanzas_baut_1_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
0,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana.,10,16,PER
1,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Esc.va,17,23,CHAR
2,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Domingo veinte y dos de [roto] y nueve,24,62,DATE
3,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Thomas de Orvera,66,82,PER
4,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana,121,126,PER
5,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Mina,137,141,CHAR
6,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,esclava,142,149,CHAR
7,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juan Joseph de Justis,159,180,PER
8,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Joseph Salcedo,197,211,PER
9,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Ana de Santiago,214,229,PER


In [ ]:
train_spacy[:2]

[('[margin]: Juana. Esc.va Domingo veinte y dos de [roto] y nueve yo Thomas de Orvera baptize, y pusse [roto] s.tos oleos a Juana de nacion Mina esclava de[roto] Juan Joseph de Justis fueron sus P.P. Joseph Salcedo y Ana de Santiago su mugger, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 16, 'PER'),
    (17, 23, 'CHAR'),
    (24, 62, 'DATE'),
    (66, 82, 'PER'),
    (121, 126, 'PER'),
    (137, 141, 'CHAR'),
    (142, 149, 'CHAR'),
    (159, 180, 'PER'),
    (197, 211, 'PER'),
    (214, 229, 'PER'),
    (263, 279, 'PER')]}),
 ('[margin]: Bernardo Esc.vo Domingo nueve de Abril de mil sietectos y diez y nueve Yo Thomas de Orvera baptize, y pusse los s.tos oleos á Bernardo negro adulto de nacion carabali esc.o de D. Juan Joseph de Justis fue su P. Andres de Morales, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 18, 'PER'),
    (19, 25, 'CHAR'),
    (26, 80, 'DATE'),
    (84, 100, 'PER'),
    (136, 144, 'PER'),
    (145, 150, 'CHAR'),
    (151, 157, 'CHA

In [ ]:
#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/test'

#load model from pretrained
nlp_model = load_model('es_core_news_md')

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,9950.058629,88.571429,6.275304,11.720227


Saved model to models\test


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,8276.646394,44.642857,5.060729,9.090909


Performance hasn't improved for 1 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,7993.203142,30.46875,7.894737,12.540193


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,71.0,7430.004824,89.211618,87.044534,88.114754


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,72.0,7437.922253,88.842975,87.044534,87.93456


Performance hasn't improved for 5 cycles...
Done training after 72 meta cycles.
